#Paquetes necesarios

In [2]:
# os is used for Ilka's linux :)
import os
os.environ["QT_QPA_PLATFORM"] = "xcb"

import cv2  
import math 

from ultralytics import YOLO

Modelos preentrenados, visualizando con las utilidades de ultralytics

In [5]:
# Carga del modelo
#model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolo11n-seg.pt') #Máscaras
model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 114.2ms
video 1/1 (frame 2/375) /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 151.1ms
video 1/1 (frame 3/375) /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/TGC23_PdH_C0056cut.mp4: 384x640 (no detections), 108.4ms
video 

Desde cámara, detección con yolo11, modelo nano. Visualización propia con OpenCV

In [6]:
# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Detecta en la imagen
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

100%|██████████| 5.35M/5.35M [00:02<00:00, 2.24MB/s]



0: 480x640 1 person, 105.6ms
Confianza ---> 0.92
Clase --> person
Speed: 1.8ms preprocess, 105.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 105.0ms
Confianza ---> 0.92
Clase --> person
Speed: 2.5ms preprocess, 105.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 111.8ms
Confianza ---> 0.92
Clase --> person
Speed: 3.4ms preprocess, 111.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 99.0ms
Confianza ---> 0.9
Clase --> person
Speed: 1.5ms preprocess, 99.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 195.6ms
Confianza ---> 0.92
Clase --> person
Speed: 6.1ms preprocess, 195.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 92.2ms
Confianza ---> 0.92
Clase --> person
Speed: 1.6ms preprocess, 92.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 p

Seguimiento. Requiere instalar lap con pip install lap

In [8]:
from collections import defaultdict
import numpy as np

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('yolo11n.pt') #Contenedores

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car"]


# Captura desde la webcam
vid = cv2.VideoCapture(0)
track_history = defaultdict(lambda: [])
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Seguimiento, con persistencia entre fotogramas
        results = model.track(img, persist=True, classes = [0,1,2])

        if 0:
            if results is not None:
                print(results[0])
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                annotated_frame = results[0].plot()
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))
                    if len(track) > 30:
                        track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                cv2.imshow("YOLO11 Tracking", annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
        

        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()




0: 480x640 1 person, 94.6ms
Speed: 1.3ms preprocess, 94.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.83
Clase --> person

0: 480x640 1 person, 87.1ms
Speed: 1.9ms preprocess, 87.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.85
Clase --> person

0: 480x640 1 person, 80.6ms
Speed: 1.5ms preprocess, 80.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.87
Clase --> person

0: 480x640 1 person, 78.3ms
Speed: 1.6ms preprocess, 78.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.87
Clase --> person

0: 480x640 1 person, 83.2ms
Speed: 2.6ms preprocess, 83.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.86
Clase --> person

0: 480x640 1 person, 80.5ms
Speed: 2.6ms preprocess, 80.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Confianza ---> 0.85
Clase --> person

0: 480x640 1 person, 

Intregración con seguimiento (tracking)
!!!!!!!!!Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [9]:
# Carga del modelo
model = YOLO('yolo11n.pt') #Contenedores
#model = YOLO('yolov11n-seg.pt') #Máscaras
#model = YOLO('yolo11n-pose.pt')  #Pose

#Para un vídeo 
filename = "TGC23_PdH_C0056cut.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/TGC23_PdH_C0056cut.mp4: 384x640 4 persons, 1 bird, 75.4ms
video 1/1 (frame 2/375) /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/TGC23_PdH_C0056cut.mp4: 384x640 4 persons, 107.7ms
video 1/1 (frame 3/375) /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/TGC23_PdH_C0056cut.mp4: 384x640 4 persons, 1 bird, 131.2ms
video 1/1

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [1]:
# Tesseract
import cv2
import platform
import pytesseract
from pytesseract import Output


if platform.system() == "Windows":
    pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
elif platform.system() == "Linux":
    pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'


# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('ocr_test.tif') 

if img is not None:
    #Convierte a RGB antes de procesar
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Texto localizado
    print(pytesseract.image_to_string(img))

    #Texto y localización en imagen de cada palabra
    d = pytesseract.image_to_data(img_rgb, output_type=Output.DICT)

    n_boxes = len(d['text'])
    for i in range(n_boxes):
        #Nivel de confianza
        if int(d['conf'][i]) > 60:
            text = d['text'][i]
            conf = d['conf'][i]
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            print(f'Texto: {text} ({conf:.2f}%)\nContenedor: {x,y,x+w,y+h}')

    cv2.imshow('img', img_rgb)
    cv2.waitKey(-1)

   

else:
    print('Error de imagen')

['eng', 'osd']
Hasta el infinito y mas alla

Texto: Hasta (96.00%)
Contenedor: (57, 95, 176, 130)
Texto: el (93.00%)
Contenedor: (194, 95, 226, 130)
Texto: infinito (91.00%)
Contenedor: (246, 95, 380, 130)
Texto: y (92.00%)
Contenedor: (396, 104, 418, 140)
Texto: mas (96.00%)
Contenedor: (435, 94, 521, 130)
Texto: alla (96.00%)
Contenedor: (538, 94, 609, 130)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jeski/sem/vc/Visi-n-por-Computador/.venv/lib/python3.10/site-packages/cv2/qt/plugins"


: 

Reconocimiento de caracteres tras instalar easyocr

In [3]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
res = reader.readtext('ocr_test.tif')

for (bbox, text, prob) in res:
    # Coordenadas en orden 
    (top_left, top_right, bottom_right, bottom_left) = bbox
    print(f'\nTexto: {text}\nProbabilidad: {prob:.2f}\nContenedor: {tuple(map(int, top_left)),tuple(map(int, bottom_right))}')


#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



Texto: Hasta el infinito y más allá
Probabilidad: 0.67
Contenedor: ((49, 85), (617, 147))


Procesar el video de ejemplo además de varios videos, hechos por nosotras o encontrados por internet
* Detecte y siga las personas y vehículos presentes
* Detecte y lea las matrículas de los vehículos presentes
* Cuente el total de cada clase
* vuelque a disco un video que visualice los resultados
* genere un archivo csv con el resultado de la detección y seguimiento. Se sugiere un formato con al
  menos los siguientes campos:
  
Se considerarán extras:
* Determine el flujo de personas y vehículos en el video de test en distintas direcciones ( vehículos 
  que dejan la imagne por la derecha, por la izquierda, etc)
* Evaluar dos alternativas para la detección de matrículas: basada en YOLO, y basada en contornos
* Anonimizar a las personas y vehículos presentes en un video
* En el caso de haberse apuntado al Autumn Campus Makeathon InnovAction Canarias, se valorará la 
    aplicación de habilidaddes adquiridas en esta práctica